In [2063]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import xgboost as xgb
import pickle


In [2064]:
# Para que se vean todas las columnas
pd.set_option('display.max_columns', None)

## Limpieza del Dataset

In [2065]:
dataset = pd.read_csv('../Datos/clientes_tarifas_modelo.csv')
dataset.head(10)

,Unnamed: 0.1,Unnamed: 0,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final
0,0,0,0,202,6,75,19,30,70,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,60.60,141.40,0.0,0.0,0.0,0.0,5.236749,24.438162,29.674911
1,1,1,0,279,23,19,58,1,99,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,2.79,276.21,0.0,0.0,0.0,0.0,0.241098,47.737374,47.978472
2,2,2,0,192,36,3,61,14,86,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,26.88,165.12,0.0,0.0,0.0,0.0,2.322835,28.537690,30.860525
3,3,3,0,114,59,23,18,56,44,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,63.84,50.16,0.0,0.0,0.0,0.0,5.516734,8.669153,14.185886
4,4,4,0,206,49,48,3,74,26,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,152.44,53.56,0.0,0.0,0.0,0.0,13.173103,9.256775,22.429877
5,5,5,0,171,35,5,60,33,67,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,56.43,114.57,0.0,0.0,0.0,0.0,4.876398,19.801133,24.677532
6,6,6,0,288,38,41,21,67,33,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,192.96,95.04,0.0,0.0,0.0,0.0,16.674638,16.425763,33.100402
7,7,7,0,120,57,11,32,74,26,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,88.80,31.20,0.0,0.0,0.0,0.0,7.673652,5.392296,13.065948
8,8,8,0,202,2,17,81,28,72,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,56.56,145.44,0.0,0.0,0.0,0.0,4.887632,25.136395,30.024028
9,9,9,0,221,39,20,41,65,35,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,143.65,77.35,0.0,0.0,0.0,0.0,12.413515,13.368400,25.781915


In [2066]:
dataset.shape

(2080, 44)

In [2067]:
dataset['tipo'].unique()

array(['DH', 'FIJO', 'PVPC', 'TRAMOS'], dtype=object)

In [2068]:
# Pasamos a tipo numerico el tipo
label_encoder = LabelEncoder()

# Aplicar LabelEncoder a la columna 'tipo'
dataset['tipo_numerico'] = label_encoder.fit_transform(dataset['tipo'])

In [2069]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0.1                    2080 non-null   int64  
 1   Unnamed: 0                      2080 non-null   int64  
 2   id_tarifa                       2080 non-null   int64  
 3   consumo_kWh                     2080 non-null   int64  
 4   %consumo_valle                  2080 non-null   int64  
 5   %consumo_llano                  2080 non-null   int64  
 6   %consumo_punta                  2080 non-null   int64  
 7   %consumo_8h                     2080 non-null   int64  
 8   %consumo_16h                    2080 non-null   int64  
 9   compania                        2080 non-null   object 
 10  tarifa                          2080 non-null   object 
 11  tp_punta_euros/kwh/dia          2080 non-null   float64
 12  tp_valle_euros/kwh/dia          20

In [2070]:
dataset

,Unnamed: 0.1,Unnamed: 0,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final,tipo_numerico
0,0,0,0,202,6,75,19,30,70,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,0.0,0.00,0.00,0.00,60.60,141.40,0.0,0.000000,0.000000,0.000000,5.236749,24.438162,29.674911,0
1,1,1,0,279,23,19,58,1,99,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,0.0,0.00,0.00,0.00,2.79,276.21,0.0,0.000000,0.000000,0.000000,0.241098,47.737374,47.978472,0
2,2,2,0,192,36,3,61,14,86,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,0.0,0.00,0.00,0.00,26.88,165.12,0.0,0.000000,0.000000,0.000000,2.322835,28.537690,30.860525,0
3,3,3,0,114,59,23,18,56,44,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,0.0,0.00,0.00,0.00,63.84,50.16,0.0,0.000000,0.000000,0.000000,5.516734,8.669153,14.185886,0
4,4,4,0,206,49,48,3,74,26,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,0.0,0.00,0.00,0.00,152.44,53.56,0.0,0.000000,0.000000,0.000000,13.173103,9.256775,22.429877,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,2075,2075,103,230,72,11,17,90,10,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,0.0,0.0,165.60,25.30,39.10,0.00,0.00,0.0,15.447499,3.286344,7.010004,0.000000,0.000000,25.743847,3
2076,2076,2076,103,255,34,62,4,51,49,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,0.0,0.0,86.70,158.10,10.20,0.00,0.00,0.0,8.087549,20.536400,1.828697,0.000000,0.000000,30.452646,3
2077,2077,2077,103,134,12,22,66,98,2,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,0.0,0.0,16.08,29.48,88.44,0.00,0.00,0.0,1.499975,3.829305,15.855877,0.000000,0.000000,21.185156,3
2078,2078,2078,103,288,11,33,56,27,73,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,0.0,0.0,31.68,95.04,161.28,0.00,0.00,0.0,2.955174,12.345221,28.914924,0.000000,0.000000,44.215318,3


In [2071]:
dataset.isnull().sum()

Unnamed: 0.1                      0
Unnamed: 0                        0
id_tarifa                         0
consumo_kWh                       0
%consumo_valle                    0
%consumo_llano                    0
%consumo_punta                    0
%consumo_8h                       0
%consumo_16h                      0
compania                          0
tarifa                            0
tp_punta_euros/kwh/dia            0
tp_valle_euros/kwh/dia            0
peaje_tran_dist_pot_punta         0
peaje_tran_dist_pot_valle         0
peaje_ener_valle_tdc              0
peaje_ener_llano_tdc              0
peaje_ener_punta_tdc              0
termino_consumo_euros/kwh/hora    0
tc_valle_euros/kwh/hora           0
tc_llano_euros/kwh/hora           0
tc_punta_euros/kwh/hora           0
tc_8h_euros/kwh/hora              0
tc_16h_euros/kwh/hora             0
mercado                           0
tipo                              0
otros                             0
alquiler_equipo_euros/dia   

In [2072]:
# Sustituir los nan por 0
dataset['margen_euros/kWh/dia'] = dataset['margen_euros/kWh/dia'].fillna(0)

In [2073]:
dataset.isnull().sum()

Unnamed: 0.1                      0
Unnamed: 0                        0
id_tarifa                         0
consumo_kWh                       0
%consumo_valle                    0
%consumo_llano                    0
%consumo_punta                    0
%consumo_8h                       0
%consumo_16h                      0
compania                          0
tarifa                            0
tp_punta_euros/kwh/dia            0
tp_valle_euros/kwh/dia            0
peaje_tran_dist_pot_punta         0
peaje_tran_dist_pot_valle         0
peaje_ener_valle_tdc              0
peaje_ener_llano_tdc              0
peaje_ener_punta_tdc              0
termino_consumo_euros/kwh/hora    0
tc_valle_euros/kwh/hora           0
tc_llano_euros/kwh/hora           0
tc_punta_euros/kwh/hora           0
tc_8h_euros/kwh/hora              0
tc_16h_euros/kwh/hora             0
mercado                           0
tipo                              0
otros                             0
alquiler_equipo_euros/dia   

In [2074]:
pd.get_dummies(dataset['mercado'], dtype='int')

,Libre,Regulado
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
2075,1,0
2076,1,0
2077,1,0
2078,1,0


In [2075]:
dataset['peaje_ener_punta_tdc'].unique()

array([0.076974])

In [2076]:
# Creamos los dummies del tipo de mercado

dt = pd.merge(dataset, pd.get_dummies(dataset['mercado'], dtype='int'), left_index=True, right_index=True)


In [2077]:
dt[dt['tipo_numerico']==0]

,Unnamed: 0.1,Unnamed: 0,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final,tipo_numerico,Libre,Regulado
0,0,0,0,202,6,75,19,30,70,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.172830,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,60.60,141.40,0.0,0.0,0.0,0.0,5.236749,24.438162,29.674911,0,1,0
1,1,1,0,279,23,19,58,1,99,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.172830,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,2.79,276.21,0.0,0.0,0.0,0.0,0.241098,47.737374,47.978472,0,1,0
2,2,2,0,192,36,3,61,14,86,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.172830,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,26.88,165.12,0.0,0.0,0.0,0.0,2.322835,28.537690,30.860525,0,1,0
3,3,3,0,114,59,23,18,56,44,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.172830,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,63.84,50.16,0.0,0.0,0.0,0.0,5.516734,8.669153,14.185886,0,1,0
4,4,4,0,206,49,48,3,74,26,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.172830,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,152.44,53.56,0.0,0.0,0.0,0.0,13.173103,9.256775,22.429877,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,175,8,129,85,13,2,35,65,Iberdrola,Plan Ahorro Inteligente,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.099999,0.168225,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,45.15,83.85,0.0,0.0,0.0,0.0,4.514955,14.105666,18.620621,0,1,0
176,176,176,8,289,85,4,11,57,43,Iberdrola,Plan Ahorro Inteligente,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.099999,0.168225,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,164.73,124.27,0.0,0.0,0.0,0.0,16.472835,20.905321,37.378156,0,1,0
177,177,177,8,174,54,1,45,75,25,Iberdrola,Plan Ahorro Inteligente,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.099999,0.168225,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,130.50,43.50,0.0,0.0,0.0,0.0,13.049869,7.317787,20.367657,0,1,0
178,178,178,8,227,20,78,2,37,63,Iberdrola,Plan Ahorro Inteligente,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.099999,0.168225,Libre,DH,0.0,0.026557,5.11,0.0,0.0,0.0,0.0,0.0,0.0,83.99,143.01,0.0,0.0,0.0,0.0,8.398916,24.057857,32.456773,0,1,0


In [2078]:
# eliminamos las columnas con valor objeto y las que no tienen correlación ya que son valores estaticos para todos, también los datos redundantes como los de los % de consumo

dt.drop(columns=['Regulado','termino_consumo_euros/kwh/hora','tc_valle_euros/kwh/hora','tc_llano_euros/kwh/hora','tc_punta_euros/kwh/hora','tc_8h_euros/kwh/hora','tc_16h_euros/kwh/hora','margen_euros/kWh/dia','peaje_tran_dist_pot_punta','peaje_tran_dist_pot_valle','tp_punta_euros/kwh/dia','tp_valle_euros/kwh/dia','otros','consumo_kWh','mercado', 'Libre', 'compania', 'tarifa','tipo','Unnamed: 0','Unnamed: 0.1','id_tarifa', 'impuesto_actual', 'alquiler_equipo_euros/dia','peaje_ener_valle_tdc','peaje_ener_llano_tdc','peaje_ener_punta_tdc','%consumo_valle','%consumo_llano','%consumo_punta','%consumo_8h','%consumo_16h'], inplace=True, errors='ignore')
dt

,bono_social_euro/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final,tipo_numerico
0,0.0,0.0,0.00,0.00,0.00,60.60,141.40,0.0,0.000000,0.000000,0.000000,5.236749,24.438162,29.674911,0
1,0.0,0.0,0.00,0.00,0.00,2.79,276.21,0.0,0.000000,0.000000,0.000000,0.241098,47.737374,47.978472,0
2,0.0,0.0,0.00,0.00,0.00,26.88,165.12,0.0,0.000000,0.000000,0.000000,2.322835,28.537690,30.860525,0
3,0.0,0.0,0.00,0.00,0.00,63.84,50.16,0.0,0.000000,0.000000,0.000000,5.516734,8.669153,14.185886,0
4,0.0,0.0,0.00,0.00,0.00,152.44,53.56,0.0,0.000000,0.000000,0.000000,13.173103,9.256775,22.429877,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,0.0,0.0,165.60,25.30,39.10,0.00,0.00,0.0,15.447499,3.286344,7.010004,0.000000,0.000000,25.743847,3
2076,0.0,0.0,86.70,158.10,10.20,0.00,0.00,0.0,8.087549,20.536400,1.828697,0.000000,0.000000,30.452646,3
2077,0.0,0.0,16.08,29.48,88.44,0.00,0.00,0.0,1.499975,3.829305,15.855877,0.000000,0.000000,21.185156,3
2078,0.0,0.0,31.68,95.04,161.28,0.00,0.00,0.0,2.955174,12.345221,28.914924,0.000000,0.000000,44.215318,3


In [2079]:
# Comprobamos la correlación y que  ya no hay valores nan

dt.corr()

,bono_social_euro/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final,tipo_numerico
bono_social_euro/dia,1.000000,-0.214917,0.191519,0.204038,0.255331,-0.069024,-0.066019,-0.214917,0.181293,0.184197,0.244173,-0.066724,-0.065746,0.075837,0.053048
consumo_unico,-0.214917,1.000000,-0.506087,-0.512578,-0.508295,-0.205563,-0.196613,1.000000,-0.489998,-0.500750,-0.502551,-0.198712,-0.195800,0.204571,-0.588843
consumo_valle,0.191519,-0.506087,1.000000,0.301371,0.301710,-0.162538,-0.155462,-0.506087,0.970132,0.294421,0.296744,-0.157121,-0.154819,0.119523,0.630890
consumo_llano,0.204038,-0.512578,0.301371,1.000000,0.278125,-0.164623,-0.157456,-0.512578,0.295552,0.978348,0.276428,-0.159137,-0.156805,0.209411,0.636629
consumo_punta,0.255331,-0.508295,0.301710,0.278125,1.000000,-0.163247,-0.156140,-0.508295,0.287804,0.267524,0.989263,-0.157807,-0.155494,0.359915,0.618913
consumo_8h,-0.069024,-0.205563,-0.162538,-0.164623,-0.163247,1.000000,0.465956,-0.205563,-0.157371,-0.160824,-0.161403,0.970980,0.467203,-0.057588,-0.428973
consumo_16h,-0.066019,-0.196613,-0.155462,-0.157456,-0.156140,0.465956,1.000000,-0.196613,-0.150520,-0.153823,-0.154376,0.445949,0.992434,0.121051,-0.410297
consumo_euros_unico,-0.214917,1.000000,-0.506087,-0.512578,-0.508295,-0.205563,-0.196613,1.000000,-0.489998,-0.500750,-0.502551,-0.198712,-0.195800,0.204571,-0.588843
consumo_euros_valle,0.181293,-0.489998,0.970132,0.295552,0.287804,-0.157371,-0.150520,-0.489998,1.000000,0.310106,0.298096,-0.152126,-0.149897,0.176089,0.611802
consumo_euros_llano,0.184197,-0.500750,0.294421,0.978348,0.267524,-0.160824,-0.153823,-0.500750,0.310106,1.000000,0.278623,-0.155464,-0.153186,0.255256,0.625478


In [2080]:

# Obtén la media de las correlaciones en la columna de referencia
mean_correlation = dt.corr()['tipo_numerico'].abs().mean()

# Filtra las columnas con correlaciones absolutas menores a la media
dropear = dt.corr().columns[dt.corr()['tipo_numerico'].abs() <= mean_correlation]

In [2081]:
dropear

Index(['bono_social_euro/dia', 'consumo_8h', 'consumo_16h', 'consumo_euros_8h',
       'consumo_euros_16h', 'precio_final'],
      dtype='object')

## XGBoost

In [2082]:
# declarar X e y
X = dt.drop(columns=['tipo_numerico'])
y = dt['tipo_numerico']

In [2083]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1453)

In [2084]:
xgb_cls = xgb.XGBClassifier(random_state = 1453)

# Los parametros todavia no están definidos
xgb_cls.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 1453,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [2085]:


# Comprobamos que el split se ha hecho correctamente

print(y_test.mean())
print(y_train.mean())
print(dt['tipo_numerico'].mean())

# Parece bastante razonable


1.8317307692307692
1.7776442307692308
1.7884615384615385


In [2086]:
xgb_cls.set_params(n_stimators=10, max_depth= 3)
xgb_cls.fit(X_train, y_train)

c:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [16:30:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_stimators" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              n_stimators=10, num_parallel_tree=None, ...)

In [2087]:
print(confusion_matrix(y_train, xgb_cls.predict(X_train)))
print(accuracy_score(y_train, xgb_cls.predict(X_train)))
print(recall_score(y_train, xgb_cls.predict(X_train), average='macro'))
print(precision_score(y_train, xgb_cls.predict(X_train), average='macro'))

[[142   0   0   0]
 [  0 748   0   0]
 [  0   0 112   0]
 [  0   0   0 662]]
1.0
1.0
1.0


In [2088]:
print(confusion_matrix(y_test, xgb_cls.predict(X_test)))
print(accuracy_score(y_test, xgb_cls.predict(X_test)))
print(recall_score(y_test, xgb_cls.predict(X_test), average='macro'))
print(precision_score(y_test, xgb_cls.predict(X_test), average='macro'))

[[ 38   0   0   0]
 [  0 172   0   0]
 [  0   0  28   0]
 [  0   0   0 178]]
1.0
1.0
1.0


## Incluimos factura

Cambiamos las columnas y generamos otras nuevas para que se adacten al dataset usado para entrenar el modelo

In [2089]:
form = pd.read_csv('../Datos/datos_tarifas.csv')
form

,Discriminacion horaria,Termino de potencia en valle,Termino de potencia en punta,Consumo en punta (kWh),Consumo en llano (kWh),Consumo en valle (kWh),Consumo total (kWh),Bono social,Total factura,Discriminación horaria
0,NaN,3.3,3.3,30.0,15.0,0.0,45.0,NO,150.0,SI
1,NaN,3.3,3.3,40.0,20.0,10.0,70.0,NO,200.0,SI
2,NaN,3.3,3.3,40.0,20.0,20.0,80.0,SI,210.0,SI
3,NaN,3.3,3.3,40.0,20.0,30.0,90.0,NO,220.0,NO


In [2090]:
form = form.drop(columns=['Discriminacion horaria','Termino de potencia en valle', 'Termino de potencia en punta'])

In [2091]:
form = form.fillna(0)

In [2092]:
form = form.rename(columns={'Consumo total (kWh)': 'consumo_unico'})
form = form.rename(columns={'Consumo en punta (kWh)': 'consumo_punta'})
form = form.rename(columns={'Consumo en llano (kWh)': 'consumo_llano'})
form = form.rename(columns={'Consumo en valle (kWh)': 'consumo_valle'})
form = form.rename(columns={'Bono social': 'bono_social'})
form = form.rename(columns={'Total factura': 'precio_final'})

In [2093]:
form['consumo_8h'] = 0
form['consumo_16h'] = 0
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h
0,30.0,15.0,0.0,45.0,NO,150.0,SI,0,0
1,40.0,20.0,10.0,70.0,NO,200.0,SI,0,0
2,40.0,20.0,20.0,80.0,SI,210.0,SI,0,0
3,40.0,20.0,30.0,90.0,NO,220.0,NO,0,0


In [2094]:
form['bono_social_euro/dia'] = form['bono_social'].apply(lambda x: dt['bono_social_euro/dia'].max() if x == 'SI' else 0)

In [2095]:
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia
0,30.0,15.0,0.0,45.0,NO,150.0,SI,0,0,0.000000
1,40.0,20.0,10.0,70.0,NO,200.0,SI,0,0,0.000000
2,40.0,20.0,20.0,80.0,SI,210.0,SI,0,0,0.033886
3,40.0,20.0,30.0,90.0,NO,220.0,NO,0,0,0.000000


In [2096]:
def actualizar_consumos(row):
    if row['consumo_punta'] > 0 and (row['consumo_valle'] == 0 or row['consumo_llano'] == 0):
        row['consumo_8h'] = row['consumo_punta']
        
        row['consumo_16h'] = row['consumo_valle'] if row['consumo_valle'] > 0 else row['consumo_llano']
        
        row['consumo_punta'] = 0
        row['consumo_valle'] = 0
        row['consumo_llano'] = 0
        row['consumo_unico'] = 0
        
    return row

# Aplicar la función a cada fila del DataFrame form
form = form.apply(actualizar_consumos, axis=1)

# Si las columnas no existen previamente, crearlas con valores NaN
form['consumo_8h'] = form.get('consumo_8h', pd.Series([None] * len(form)))
form['consumo_16h'] = form.get('consumo_16h', pd.Series([None] * len(form)))

In [2097]:
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia
0,0.0,0.0,0.0,0.0,NO,150.0,SI,30.0,15.0,0.000000
1,40.0,20.0,10.0,70.0,NO,200.0,SI,0.0,0.0,0.000000
2,40.0,20.0,20.0,80.0,SI,210.0,SI,0.0,0.0,0.033886
3,40.0,20.0,30.0,90.0,NO,220.0,NO,0.0,0.0,0.000000


In [2098]:

form['consumo_unico'] = form.apply(lambda row: row['consumo_unico'] if row['Discriminación horaria'] == 'NO' else 0, axis=1)

In [2099]:
# Si tienen discrepancia horaria se rellenan con los valores ya existentes si NO tienen discrepancia se rellenan con un cero

form['consumo_punta'] = form.apply(lambda row: row['consumo_punta'] if row['Discriminación horaria'] == 'SI' else 0, axis=1)
form['consumo_valle'] = form.apply(lambda row: row['consumo_valle'] if row['Discriminación horaria'] == 'SI' else 0, axis=1)
form['consumo_llano'] = form.apply(lambda row: row['consumo_llano'] if row['Discriminación horaria'] == 'SI' else 0, axis=1)

In [2100]:
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia
0,0.0,0.0,0.0,0.0,NO,150.0,SI,30.0,15.0,0.000000
1,40.0,20.0,10.0,0.0,NO,200.0,SI,0.0,0.0,0.000000
2,40.0,20.0,20.0,0.0,SI,210.0,SI,0.0,0.0,0.033886
3,0.0,0.0,0.0,90.0,NO,220.0,NO,0.0,0.0,0.000000


In [2101]:
#form.at[form.index[-1], 'consumo_unico'] = 0

In [2102]:
#form.at[form.index[-1], 'consumo_llano'] = 0

In [2103]:
#form.at[form.index[-1], 'consumo_punta'] = 30

In [2104]:
#form.at[form.index[-1], 'consumo_valle'] = 30

In [2105]:
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia
0,0.0,0.0,0.0,0.0,NO,150.0,SI,30.0,15.0,0.000000
1,40.0,20.0,10.0,0.0,NO,200.0,SI,0.0,0.0,0.000000
2,40.0,20.0,20.0,0.0,SI,210.0,SI,0.0,0.0,0.033886
3,0.0,0.0,0.0,90.0,NO,220.0,NO,0.0,0.0,0.000000


In [2106]:
dt.head(1)

,bono_social_euro/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final,tipo_numerico
0,0.0,0.0,0.0,0.0,0.0,60.6,141.4,0.0,0.0,0.0,0.0,5.236749,24.438162,29.674911,0


In [2107]:
# colocamos la media de los valores en euros para tener algún valor con sentido con respecto al dataset con el que hemos entrenado el modelo

mean_consumo_euros_unico = dt.loc[dt['consumo_euros_unico'] > 0, 'consumo_euros_unico'].mean()
form['consumo_euros_unico'] = form['consumo_unico'].apply(lambda x: mean_consumo_euros_unico if x > 0 else 0)

mean_consumo_euros_punta = dt.loc[dt['consumo_euros_punta'] > 0, 'consumo_euros_punta'].mean()
form['consumo_euros_punta'] = form['consumo_punta'].apply(lambda x: mean_consumo_euros_punta if x > 0 else 0)

mean_consumo_euros_llano = dt.loc[dt['consumo_euros_llano'] > 0, 'consumo_euros_llano'].mean()
form['consumo_euros_llano'] = form['consumo_llano'].apply(lambda x: mean_consumo_euros_llano if x > 0 else 0)

mean_consumo_euros_valle = dt.loc[dt['consumo_euros_valle'] > 0, 'consumo_euros_valle'].mean()
form['consumo_euros_valle'] = form['consumo_valle'].apply(lambda x: mean_consumo_euros_valle if x > 0 else 0)

mean_consumo_euros_16h = dt.loc[dt['consumo_euros_16h'] > 0, 'consumo_euros_16h'].mean()
form['consumo_euros_16h'] = form['consumo_16h'].apply(lambda x: mean_consumo_euros_16h if x > 0 else 0)

mean_consumo_euros_8h = dt.loc[dt['consumo_euros_8h'] > 0, 'consumo_euros_8h'].mean()
form['consumo_euros_8h'] = form['consumo_8h'].apply(lambda x: mean_consumo_euros_8h if x > 0 else 0)
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,bono_social,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia,consumo_euros_unico,consumo_euros_punta,consumo_euros_llano,consumo_euros_valle,consumo_euros_16h,consumo_euros_8h
0,0.0,0.0,0.0,0.0,NO,150.0,SI,30.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,18.597894,9.692422
1,40.0,20.0,10.0,0.0,NO,200.0,SI,0.0,0.0,0.000000,0.000000,13.184334,10.203053,7.642336,0.000000,0.000000
2,40.0,20.0,20.0,0.0,SI,210.0,SI,0.0,0.0,0.033886,0.000000,13.184334,10.203053,7.642336,0.000000,0.000000
3,0.0,0.0,0.0,90.0,NO,220.0,NO,0.0,0.0,0.000000,28.217803,0.000000,0.000000,0.000000,0.000000,0.000000


In [2108]:
form['precio_final'] = form['consumo_euros_punta'] + form['consumo_euros_llano'] + form['consumo_euros_valle'] + form['consumo_euros_unico']+ form['consumo_euros_16h']+ form['consumo_euros_8h']

In [2109]:
form.drop(columns=['bono_social'], inplace=True)

In [2110]:
form.fillna(0, inplace=True)

In [2111]:
form

,consumo_punta,consumo_llano,consumo_valle,consumo_unico,precio_final,Discriminación horaria,consumo_8h,consumo_16h,bono_social_euro/dia,consumo_euros_unico,consumo_euros_punta,consumo_euros_llano,consumo_euros_valle,consumo_euros_16h,consumo_euros_8h
0,0.0,0.0,0.0,0.0,28.290317,SI,30.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,18.597894,9.692422
1,40.0,20.0,10.0,0.0,31.029724,SI,0.0,0.0,0.000000,0.000000,13.184334,10.203053,7.642336,0.000000,0.000000
2,40.0,20.0,20.0,0.0,31.029724,SI,0.0,0.0,0.033886,0.000000,13.184334,10.203053,7.642336,0.000000,0.000000
3,0.0,0.0,0.0,90.0,28.217803,NO,0.0,0.0,0.000000,28.217803,0.000000,0.000000,0.000000,0.000000,0.000000


In [2112]:
form = form.reindex(columns=['bono_social_euro/dia', 'consumo_unico', 'consumo_valle', 'consumo_llano', 'consumo_punta', 'consumo_8h', 'consumo_16h', 'consumo_euros_unico', 'consumo_euros_valle', 'consumo_euros_llano', 'consumo_euros_punta', 'consumo_euros_8h', 'consumo_euros_16h', 'precio_final'])

form

,bono_social_euro/dia,consumo_unico,consumo_valle,consumo_llano,consumo_punta,consumo_8h,consumo_16h,consumo_euros_unico,consumo_euros_valle,consumo_euros_llano,consumo_euros_punta,consumo_euros_8h,consumo_euros_16h,precio_final
0,0.000000,0.0,0.0,0.0,0.0,30.0,15.0,0.000000,0.000000,0.000000,0.000000,9.692422,18.597894,28.290317
1,0.000000,0.0,10.0,20.0,40.0,0.0,0.0,0.000000,7.642336,10.203053,13.184334,0.000000,0.000000,31.029724
2,0.033886,0.0,20.0,20.0,40.0,0.0,0.0,0.000000,7.642336,10.203053,13.184334,0.000000,0.000000,31.029724
3,0.000000,90.0,0.0,0.0,0.0,0.0,0.0,28.217803,0.000000,0.000000,0.000000,0.000000,0.000000,28.217803


## PREDICCIÓN

In [2117]:
last_row = form.iloc[[-1]]

In [2118]:
last_row_array = last_row.values
prediccion = xgb_cls.predict(last_row_array)

In [2119]:
prediccion

array([1], dtype=int64)

In [2120]:
if prediccion == 1:
    print("La predicción es: Tarifa fija")
elif prediccion == 2:
    print("La predicción es: PVPC")
elif prediccion == 3:
    print("La predicción es: Tres tramos de tiempo")
elif prediccion == 0:
    print("La predicción es: Dos tramos de tiempo")

La predicción es: Tarifa fija
